# Trabalho 01 – Implementação Em Python Do Método De Gauss-seidel

---



## Alunos

João Guilherme Vargas e Marco Antônio

## Descrição do Trabalho

`Apresente o código **COMENTADO** em **PYTHON** do **MÉTODO DE GAUSS-SEIDEL** e utilize este código para resolver o problema apresentado. <br>
Você deve enviar um arquivo único no formato [.txt] para que o professor possa testá-lo no ambiente do Google Colab <br>

Encontre a solução do Sistema Linear:
$$
\begin{cases}
7x_1+1x_2-1x_3+4x_4=-10 \\
1x_1-5x_2+1x_3-2x_4=2 \\
1x_1+0x_2+3x_3-1x_4=11 \\
3x_1+1x_2-3x_3+8x_4=-24
\end{cases}
$$



Pelo **Método de Jacobi** com $\varepsilon \leq 0.0005$ e $ [x^0]^T = \begin{bmatrix} -0.3 & 1.3 & 2.8 & -2.3 \end{bmatrix}$ <br>
(utilize os testes de convergência e trocas de linha se necessário) <br>

Implementar também uma forma de avaliar a qualidade do ajuste, através do vetor resíduo.

# 1. Introdução

## 1.1 Preparando o Ambiente

Vamos preparar o ambiente de execução com as informações dadas pela descrição, além das bibliotecas necessárias. Só é necessário rodar os blocos de código abaixo uma vez cada.

### 1.1.1 Importar bibliotecas

In [ ]:
import numpy as np

### 1.1.2 Definir as variáveis

In [ ]:
m_A = np.array([
  [7,  1,  -1,  4],
  [1, -5,  1,  -2],
  [1,  0,  3,  -1],
  [3,  1,  -3,  8]]
  , dtype=np.float64) # Matriz A

v_b = np.array([-10, 2, 11, -24], dtype=np.float64) # Vetor b

tolerancia = 0.0005  # Tolerância de erro

x0 = np.array([-0.3, 1.3, 2.8, -2.3], dtype=np.float64)  # Chute inicial para a solução

### 1.1.3 Definir as funções auxiliares

In [ ]:
# Função que calcula o erro absoluto entre os vetores verdadeiro e aproximado
def erro_absoluto(vetor_verdadeiro, vetor_aproximado, norma="euclidiana"):
  diferenca = np.array(vetor_verdadeiro) - np.array(vetor_aproximado)

  if norma == "euclidiana":
    return np.linalg.norm(diferenca, ord=2)
  elif norma == "infinita":
    return np.linalg.norm(diferenca, ord=np.inf)
  else:
    raise ValueError("Norma inválida. Use 'euclidiana' ou 'infinita'.")


# Função que calcula o erro relativo entre os vetores verdadeiro e aproximado
def erro_relativo(vetor_verdadeiro, vetor_aproximado, norma="euclidiana"):
  erro_abs = erro_absoluto(vetor_verdadeiro, vetor_aproximado, norma)

  norma_verdadeiro = np.linalg.norm(vetor_verdadeiro, ord=2 if norma == "euclidiana" else np.inf)
  if norma_verdadeiro == 0:
    raise ZeroDivisionError("A norma do vetor verdadeiro é zero, não é possível calcular o erro relativo.")
  return erro_abs / norma_verdadeiro


# Função que verifica se a matriz A é diagonal dominante
def tem_diagonal_dominante(A):
  for i in range(len(A)):
    soma_linha = sum(abs(A[i][j]) for j in range(len(A[i])) if i != j)
    if abs(A[i][i]) <= soma_linha:
      return False
  return True


# Função que tenta tornar a matriz A diagonal dominante, se necessário
def tornar_diagonal_dominante(A, b):
  n = len(A)
  A = A.copy()
  b = b.copy()
  for i in range(n):
    max_index = max(range(i, n), key=lambda k: abs(A[k][i]))
    if i != max_index:
      A[[i, max_index]] = A[[max_index, i]]
      b[i], b[max_index] = b[max_index], b[i]
  return A, b

def calcular_residuo(A, b, x):
  return b - np.dot(A, x)

## 1.2 Métodos Iterativos

Um conjunto de procedimentos utilizados para transformar um modelo matemático num problema numérico ou um conjunto de procedimentos para resolver um problema numérico. Caracterizado por envolver os seguintes elementos constitutivos:


1. Aproximação inicial: <br>
Consiste em uma primeira aproximação para a solução desejada do problema numérico
2. Equação de recorrência: <br>
Equação por meio da qual, partindo-se da tentativa inicial, são realizadas as iterações para a solução desejada
3. Critério de parada: <br>
É o instrumento por meio do qual o procedimento iterativa é finalizado. Maiores detalhes sobre métodos iterativos serão discutidos ao longo dos capítulos



# 2. Método de Jacobi

## 2.1 Descrição

Dado um sistema linear com $a_{ii} \neq 0$:
$$
\begin{cases}
a_{11}x_1 + a_{12}x_2 + ... + a_{1n}x_n & = b_1\\
a_{21}x_1 + a_{22}x_2 + ... + a_{2n}xn & = b_2 \\
... & = ... \\
a_{n1}x_1 + a_{n2}x_2 + ... + a_{nn}x_n & = bn
\end{cases}
$$

Isolando cada $x_n$ das equações, obtem-se:
$$
\begin{cases}
x_1 = \frac{b_1 - a_{11}x_1 + a_{12}x_2 + ... + a_{1n}x_{n}}{a_{11n}} \\
x_2 = \frac{b_2 - a_{21}x_1 + a_{23}x_3 + ... + a_{2n}x_{n}}{a_{22}} \\
.. \\
x_n = \frac{b_n - a_{n1}x_1 + a_{n2}x_2 + ... + a_{nn-1}x_{n-1}}{a_{nn}}
\end{cases}
$$

Reescrevendo, tem-se:
$$
x = Fx + d \\
x =
\begin{bmatrix}
0 & -\frac{a_{12}}{a_{11}} & -\frac{a_{13}}{a_{11}} & ... & -\frac{a_{1n}}{a_{11}} \\
-\frac{a_{21}}{a_{22}} & 0 & -\frac{a_{23}}{a_{22}} & ... & -\frac{a_{2n}}{a_{22}} \\
\vdots & \vdots & \vdots & \vdots & \vdots \\
-\frac{a_{n1}}{a_{nn}} & -\frac{a_{n2}}{a_{nn}} & -\frac{a_{n3}}{a_{nn}} & ... & 0
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{bmatrix}
+
\begin{bmatrix}
\frac{b_{1}}{a_{11}} \\
\frac{b_{2}}{a_{22}} \\
\vdots \\
\frac{b_{n}}{a_{nn}} \\
\end{bmatrix}
$$

Portanto, através de Jacobi tem-se que: <br>
1. Escolhe-se uma aproximação inicial $x^{(0)} = (x_{1}^{(0)}, x_{2}^{(0)}, ..., x_{n}^{(0)})
$ \\
2. Geram-se sucessivas $x^{(k)}$ a partir da iteração $x^{(k+1)} = Fx^{(k)}+d$ para $k = 0, 1, 2, ...$ \\
3. Continua-se iterando até o critério de parada seja alcançado.

### 2.1.1 Critério de Parada

Como critério de parada podemos usar os erros absoluto e relativo, com as normas $1$, $2$ ou $\infty$.

* $||x^{(k)} = x^{(x-1)}|| < \varepsilon$
* $\frac{||x^{(k)} = x^{(x-1)}||}{||x^{(k)}||} < \varepsilon$
* $k > k_{max}$ em que $k_{max}$ é o número máximo de iterações.



### 2.1.2 Critério de Convergencia

Dado um sistema $Ax = b$, é condição suficiente para a convergência do método iterativo de Jacobi
que a matriz dos coeficientes A seja diagonal estritamente dominante por linhas, ou seja:
$$
|a_{ii}| > \sum_{j=1, j \neq i}^{n}|a_{ij}|, i = 1, 2, ... , n
$$

## 2.2 Implementação

### 2.2.1 Métodos

In [ ]:
def metodo_jacobi(A, b, x0, tolerancia, norma="euclidiana", max_iter=500, num_casas=6):
  A, b = tornar_diagonal_dominante(A, b)
  if not tem_diagonal_dominante(A):
    raise ValueError("Não foi possível tornar a matriz diagonal dominante. O método pode não convergir.")

  vetor_resolvido = np.linalg.solve(A, b)
  x = x0
  num_linhas = len(A)

  print(f"Iteração 0: x = {x}")

  for it in range(max_iter):
    x_new = np.zeros_like(x)  # Cria um vetor de zeros para armazenar a nova solução
    for i in range(num_linhas):
      # Calcula a nova solução para x[i] com base nos valores anteriores de x
      s = sum(A[i][j] * x[j] for j in range(num_linhas) if i != j)
      x_new[i] = (b[i] - s) / A[i][i]  # Fórmula do método de Jacobi

    err_abs = erro_absoluto(x_new, x, norma) # Calcula o erro relativo entre as soluções
    print(f"Iteração {it + 1}: x = {x_new}")
    print(f"Erro absoluto: {err_abs:.6f} \n")

    if err_abs <= tolerancia:
      print(f"\nConvergiu em {it + 1} iterações")
      return x_new

    x = x_new


  print(f"Não convergiu após {max_iter} iterações.")

print("=== Método de Jacobi ===")
solucao_jacobi = metodo_jacobi(m_A, v_b, x0, tolerancia, norma="euclidiana", max_iter=500)
residuo = calcular_residuo(m_A, v_b, solucao_gs)
print(f"Solução aproximada (Jacobi): {solucao_jacobi}")
print(f"Solução exata: {np.linalg.solve(m_A, v_b)}")
print(f"Erro relativo: {(erro_relativo(np.linalg.solve(m_A, v_b), solucao_jacobi)*100)}%")
print(f"Vetor resíduo: {residuo}")

=== Método de Jacobi ===
Iteração 0: x = [-0.3  1.3  2.8 -2.3]
Iteração 1: x = [ 0.1   1.02  3.   -2.  ]
Erro absoluto: 0.606960 

Iteração 2: x = [-2.85714286e-03  1.02000000e+00  2.96666667e+00 -2.04000000e+00]
Erro absoluto: 0.115285 

Iteração 3: x = [ 0.0152381   1.0087619   2.98761905 -2.01392857]
Erro absoluto: 0.039654 

Iteração 4: x = [ 0.00493878  1.00614286  2.99027778 -2.01145238]
Erro absoluto: 0.011231 

Iteração 5: x = [ 0.00427778  1.00362426  2.99453628 -2.00626573]
Erro absoluto: 0.007198 

Iteração 6: x = [ 2.28213476e-03  1.00226910e+00  2.99648550e+00 -2.00410609e+00]
Erro absoluto: 0.003779 

Iteração 7: x = [ 1.52010987e-03  1.00139596e+00  2.99787059e+00 -2.00245738e+00]
Erro absoluto: 0.002445 

Iteração 8: x = [ 9.00590745e-04  1.00086109e+00  2.99867417e+00 -2.00154307e+00]
Erro absoluto: 0.001467 

Iteração 9: x = [ 5.69334467e-04  1.00053218e+00  2.99918545e+00 -2.00094254e+00]
Erro absoluto: 0.000916 

Iteração 10: x = [ 3.46206377e-04  1.00032797e+00  2.

# 3. Método de Gauss-Seidel

O método de Gauss-Seidel é um método iterativo para resolução de sistemas lineares. As equações do método se baseam nos resultados anteriores e, diferente do método de Jacobi, também se basea nos resultados que estão sendo obtidos nessa iteração. No caso de um sistema com quatro variáveis (como o sistema que será testado), ele segue as seguintes fórmulas.

$$
x_1^{k+1}= \frac{b_1-(a_{12}x_2^{k}+a_{13}x_3^k+a_{14}x_{4}^{k})}{a_{11}} \\
x_2^{k+1}=\frac{b_2-(a_{21}x_1^{k+1}+a_{23}x_3^k+a_{24}x_4^{k})}{a_{22}} \\
x_3^{k+1}=\frac{b_3-(a_{31}x_1^{k+1}+a_{32}x_2^{k+1}+a_{34}x_4^{k})}{a_{33}} \\
x_4^{k+1}=\frac{b_4-(a_{41}x_1^{k+1}+a_{42}x_2^{k+1}+a_{43}x_3^{k+1})}{a_{44}}
$$

$$ {\displaystyle x_{i}^{(k+1)}={\frac {1}{a_{ii}}}\left(b_{i}-\sum _{j<i}a_{ij}x_{j}^{(k+1)}-\sum _{j>i}a_{ij}x_{j}^{(k)}\right),\,i=1,2,\ldots ,n.}$$


In [ ]:
def metodo_gauss_seidel(A, b, x0, tolerancia, norma="euclidiana", max_iter=500):
  if not tem_diagonal_dominante(A):
    raise ValueError("A matriz não é diagonal dominante. O método pode não convergir.")

  x = x0
  num_linhas = len(A)

  print(f"Iteração 0: x = {x}\n")

  for it in range(max_iter):
    x_new = np.copy(x)  # Copia o vetor atual para atualizar os valores

    for i in range(num_linhas):
      s1 = sum(A[i][j] * x_new[j] for j in range(i))  # Soma com os valores já atualizados
      s2 = sum(A[i][j] * x[j] for j in range(i + 1, num_linhas))  # Soma com os valores antigos
      x_new[i] = (b[i] - s1 - s2) / A[i][i]  # Atualiza x[i]

    err_abs = erro_absoluto(x_new, x, norma)  # Calcula o erro absoluto
    print(f"Iteração {it + 1}: x = {x_new}")
    print(f"Erro absoluto: {err_abs:.6f} \n")

    if err_abs <= tolerancia:
      print(f"\nConvergiu em {it + 1} iterações")
      return x_new

    x = x_new  # Atualiza o vetor x para a próxima iteração

  print(f"Não convergiu após {max_iter} iterações.")
  return x

print("\n=== Método de Gauss-Seidel ===")
solucao_gs = metodo_gauss_seidel(m_A, v_b, x0, tolerancia, norma="euclidiana", max_iter=500)
residuo = calcular_residuo(m_A, v_b, solucao_gs)
print(f"Solução aproximada (Gauss-Seidel): {solucao_gs}")
print(f"Solução exata: {np.linalg.solve(m_A, v_b)}")
print(f"Erro relativo: {(erro_relativo(np.linalg.solve(m_A, v_b), solucao_gs) * 100)}%")
print(f"Vetor resíduo: {residuo}")
print(f"Norma do resíduo: {np.linalg.norm(residuo)}")


=== Método de Gauss-Seidel ===
Iteração 0: x = [-0.3  1.3  2.8 -2.3]

Iteração 1: x = [ 0.1         1.1         2.86666667 -2.1       ]
Erro absoluto: 0.494413 

Iteração 2: x = [ 0.02380952  1.01809524  2.95873016 -2.02666667]
Erro absoluto: 0.162379 

Iteração 3: x = [ 0.00675737  1.00376417  2.98885865 -2.00718254]
Erro absoluto: 0.042232 

Iteração 4: x = [ 1.97494871e-03  1.00103974e+00  2.99694750e+00 -2.00201526e+00]
Erro absoluto: 0.011065 

Iteração 5: x = [ 5.66971838e-04  1.00030900e+00  2.99913926e+00 -2.00057402e+00]
Erro absoluto: 0.003066 

Iteração 6: x = [ 1.60904314e-04  1.00008964e+00  2.99975503e+00 -2.00016341e+00]
Erro absoluto: 0.000872 

Iteração 7: x = [ 4.55748350e-05  1.00002548e+00  2.99993034e+00 -2.00004640e+00]
Erro absoluto: 0.000249 


Convergiu em 7 iterações
Solução aproximada (Gauss-Seidel): [ 4.55748350e-05  1.00002548e+00  2.99993034e+00 -2.00004640e+00]
Solução exata: [ 2.22044605e-16  1.00000000e+00  3.00000000e+00 -2.00000000e+00]
Erro relativo